In [1]:
import os
import torch

from pyro.distributions import Uniform

from pyciemss.ODE.base import PetriNetODESystem, GaussianNoisePetriNetODESystem
from pyciemss.ODE.events import Event, ObservationEvent, LoggingEvent, StartEvent, DynamicStopEvent
import pyciemss

In [6]:
STARTERKIT_PATH = "test/models/starter_kit_examples/"
MIRA_PATH = "test/models/evaluation_examples/"

filename = "CHIME-SIR/model_petri.json"
filename = os.path.join(STARTERKIT_PATH, filename)

model = GaussianNoisePetriNetODESystem.from_mira(filename)
model.load_start_event(0.0, {"S": 0.9, "I": 0.1, "R": 0.0})
model.load_logging_events(torch.linspace(1, 10, 10))

solution = model()
solution

# See that the solution has the correct number of events.
assert len(solution["I"]) == 11

In [11]:
# Remove the logging events.
model.delete_logging_events()

# Add observation events.
# TODO: implement a pyciemss.condition operation that returns a model with observation_events
model._observation_events = [ObservationEvent(1.3, "R", torch.tensor(1.0)), ObservationEvent(2.3, "I", torch.tensor(0.2))]
model._construct_static_events()
solution

{'I': tensor([0.1000, 0.0687, 0.0485]),
 'R': tensor([0.0000, 0.1182, 0.1811]),
 'S': tensor([0.9000, 0.8132, 0.7704])}

In [ ]:
model.delete_observ

In [5]:
from pyro.infer.autoguide import AutoNormal
from pyro.poutine import block
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import pyro

guide = AutoNormal(model)
optim = Adam({'lr': 0.03})
loss_f = Trace_ELBO(num_particles=1)
verbose = True

svi = SVI(model, guide, optim, loss=loss_f)

pyro.clear_param_store()

for j in range(100):
    # calculate the loss and take a gradient step
    # Passing a data argument to svi.step() will condition the model on the data.
    loss = svi.step()
    if verbose:
        if j % 25 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss))

[iteration 0001] loss: 7.3121
[iteration 0026] loss: 5.6594
[iteration 0051] loss: 4.4821
[iteration 0076] loss: 2.8369


In [6]:
model.a_beta

tensor(0.9956, grad_fn=<ExpandBackward0>)

In [7]:
guide()

{'a_beta': tensor(1.0105, grad_fn=<ExpandBackward0>),
 'a_gamma': tensor(0.9744, grad_fn=<ExpandBackward0>)}

In [ ]:
# SCAFFOLDING FOR DYNAMIC EVENT HANDLING BELOW

import pyro
from torch import Tensor
from torchdiffeq import odeint_event, odeint

Time = Tensor
State = tuple[Tensor, ...]

class BaseODEModel(pyro.nn.PyroModule):

    def __init__(self, static_events: list[Event]):
        super().__init__()
        # This is a list of events that are always sorted by time.
        # TODO: probably pre-sort this list just in case.
        self.static_events = static_events

    def deriv(self, t: Time, state: State) -> State:
        raise NotImplementedError

    @pyro.nn.pyro_method
    def param_prior(self):
        raise NotImplementedError

    @pyro.nn.pyro_method
    def initial_conditions_prior(self):
        raise NotImplementedError

    # @pyro.nn.pyro_method
    # def observation_model(self, state: State, tspan: Time, ?) -> ?: …

    def solve(self, initial_state: State, initial_time: Time) -> tuple[State, State]:
        
        current_state = initial_state
        current_time = initial_time

        solution = torch.tensor([])

        for i, static_event in enumerate(self.static_events):
            
            # TODO: change below
            # Note: Immediate goal is to get self.solve() to generate ode solutions that stop at the event times.
            # Note: Next we will add log likelihood at those events using an observation model.

            # Note: each static_event is an Event object, which has a forward() method that returns a Tensor.
            # TODO: Chad, could you please make this actually make sense.
            event_time, event_solution = odeint(self.deriv, current_state, current_time, event_fn=static_event)
    
            # TODO: Add log likelihood for event_solution.
            # Note: This will be done by adding an observation model to the ODE model and calling it with the event_solution as an argument.

            solution = torch.cat([solution, event_solution], dim=-1)
            # current = event_solution[-1]


    def forward(self, initial_state: State, initial_time: Time) -> tuple[State, State]:
        # Sample parameters from the prior. These parameters are generated as attributes of the model.
        self.param_prior()

        # TODO: Sample initial conditions from the prior instead of taking them as input.

        # Solve the ODE, taking into account any intervention and conditioning events.
        return self.solve(initial_state, initial_time)




In [ ]:
event_time, event_solution = odeint(self.deriv, current_state, current_time, event_fn=static_event)

In [ ]:

# TODO: Chad, initialize a stop event here.
# stop_event = ...

# TODO: Chad, create a model the subclasses BaseODEModel and has all of the above missing methods implemented.
# deriv, param_prior, initial_conditions_prior, observation_model
# class ActualODEModel(BaseODEModel):
#   ...

# TODO: Chad, instantiate the model here.
# model = ActualODEModel([stop_event])

# TODO: Chad, Call model.forward() here.
# solution, _ = model(initial_state, tspan)

In [7]:
observations = [ObservationEvent(torch.tensor([2.1]), {"I": torch.tensor([0.1])})]


In [8]:
conditioned_model = pyciemss.condition(model, observations)

AttributeError: module 'pyciemss' has no attribute 'condition'